In [5]:
# =============================================================
# 📦 SAFE LAKEHOUSE EXPORT (Pandas-based Local Writer)
# Works inside Microsoft Fabric, Synapse, or Databricks
# =============================================================

from pyspark.sql import SparkSession
import os, shutil, pandas as pd

class SafeLakehouseExporter:
    def __init__(self, spark: SparkSession, output_dir="/tmp/lakehouse_export_local"):
        self.spark = spark
        self.output_dir = output_dir
        os.makedirs(self.output_dir, exist_ok=True)

    def list_tables(self, databases):
        """List all tables across provided databases."""
        tables = []
        for db in databases:
            try:
                for t in self.spark.catalog.listTables(db):
                    tables.append(f"{db}.{t.name}")
            except Exception as e:
                print(f"⚠️ Could not access {db}: {e}")
        return tables

    def export_table(self, table_name):
        """Export a single Delta table using pandas conversion."""
        try:
            print(f"📤 Exporting {table_name} ...")
            df = self.spark.table(table_name)
            pdf = df.limit(50000).toPandas()  # limit safeguard (you can adjust)
            csv_path = os.path.join(self.output_dir, f"{table_name.replace('.', '_')}.csv")
            pdf.to_csv(csv_path, index=False)
            print(f"✅ Saved {csv_path} ({len(pdf)} rows)")
        except Exception as e:
            print(f"⚠️ Skipping {table_name}: {e}")

    def export_all(self, databases=["ParkingDataLakehouse", "MirroredData"]):
        """Export all Lakehouse tables to local CSVs."""
        print("🚀 Starting Lakehouse Export (Safe Mode - Local Write)...")

        tables = self.list_tables(databases)
        if not tables:
            print("⚠️ No tables found.")
            return None

        for t in tables:
            self.export_table(t)

        # Zip for easy download
        zip_path = f"{self.output_dir}.zip"
        shutil.make_archive(self.output_dir, "zip", self.output_dir)
        print(f"🏁 Export completed successfully → {zip_path}")
        return zip_path


# -------------------------------------------------------------
# ✅ Optional Direct Run
# -------------------------------------------------------------
if __name__ == "__main__":
    spark = SparkSession.builder.appName("SafeLakehouseExporter").getOrCreate()
    exporter = SafeLakehouseExporter(spark)
    zip_path = exporter.export_all()
    print(f"📦 All tables exported → {zip_path}")


StatementMeta(, dd499c23-7591-40fb-b9fd-cd91ddf660f8, 7, Finished, Available, Finished)

🚀 Starting Lakehouse Export (Safe Mode - Local Write)...
📤 Exporting ParkingDataLakehouse.parkingsensordata ...
✅ Saved /tmp/lakehouse_export_local/ParkingDataLakehouse_parkingsensordata.csv (2 rows)
📤 Exporting ParkingDataLakehouse.trafficcameradata ...
✅ Saved /tmp/lakehouse_export_local/ParkingDataLakehouse_trafficcameradata.csv (2 rows)
📤 Exporting ParkingDataLakehouse.yoloprocesseddata ...
✅ Saved /tmp/lakehouse_export_local/ParkingDataLakehouse_yoloprocesseddata.csv (2 rows)
📤 Exporting MirroredData.parkingsensordata ...
✅ Saved /tmp/lakehouse_export_local/MirroredData_parkingsensordata.csv (2 rows)
📤 Exporting MirroredData.trafficcameradata ...
✅ Saved /tmp/lakehouse_export_local/MirroredData_trafficcameradata.csv (2 rows)
📤 Exporting MirroredData.yoloprocesseddata ...
✅ Saved /tmp/lakehouse_export_local/MirroredData_yoloprocesseddata.csv (2 rows)
🏁 Export completed successfully → /tmp/lakehouse_export_local.zip
📦 All tables exported → /tmp/lakehouse_export_local.zip


In [6]:
import shutil
shutil.copy("/tmp/lakehouse_export_local.zip", "/lakehouse/default/Files/lakehouse_export_local.zip")


StatementMeta(, dd499c23-7591-40fb-b9fd-cd91ddf660f8, 8, Finished, Available, Finished)

'/lakehouse/default/Files/lakehouse_export_local.zip'